In [11]:
import numpy, pandas
import os

In [12]:
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import oauth2
import random
from functools import reduce

import json, os
import traceback
import matplotlib.pyplot as plt
%matplotlib inline

import wget

In [13]:
from groupify.app.spotify_utils import *
from groupify.app.ml_utils import *
from groupify.app.feature_engineering_utils import *

In [14]:
redirect_uri = 'https://example.com'

In [44]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

import groupify.app.psecrets as psecrets

try: 
    cid = psecrets.client_id
    secret = psecrets.secret
    userdata = json.load(open('groupify/app/json/userdata.json', 'r'))
    username = list(userdata.keys())[0]
    scope = 'user-top-read user-read-recently-played user-follow-read playlist-modify-public user-library-read playlist-read-collaborative'
    token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

    if token:
        sp = spotipy.Spotify(auth=token)
    else:
        print("Can't get token for", username)
except: 
    print(traceback.format_exc(5))


In [45]:
list_of_genres = pandas.read_html('https://everynoise.com/everynoise1d.cgi?scope=all', index_col=0)
list_of_genres = list_of_genres[0].replace('☊', '')
list_of_genres = list_of_genres.values.tolist()
list_of_genres = [a[1] for a in list_of_genres]
list_of_genres

['pop',
 'dance pop',
 'rap',
 'rock',
 'hip hop',
 'latin',
 'pop rap',
 'trap latino',
 'trap',
 'modern rock',
 'edm',
 'reggaeton',
 'pop dance',
 'post-teen pop',
 'latin pop',
 'electropop',
 'pop rock',
 'classic rock',
 'tropical house',
 'r&b',
 'mellow gold',
 'musica mexicana',
 'soft rock',
 'permanent wave',
 'album rock',
 'melodic rap',
 'southern hip hop',
 'alternative metal',
 'k-pop',
 'urban contemporary',
 'canadian pop',
 'uk pop',
 'electro house',
 'contemporary country',
 'indie pop',
 'alternative rock',
 'alt z',
 'gangster rap',
 'nueva musica mexicana',
 'hard rock',
 'nu metal',
 'country',
 'ranchera',
 'rock en espanol',
 'tropical',
 'latin hip hop',
 'desi pop',
 'art pop',
 'neo mellow',
 'underground hip hop',
 'adult standards',
 'atl hip hop',
 'modern alternative rock',
 'french hip hop',
 'country road',
 'folk rock',
 'arrocha',
 'chicago rap',
 'filmi',
 'new wave pop',
 'pop urbaine',
 'german hip hop',
 'art rock',
 'soul',
 'country rock',
 

In [17]:
num_playlists=3
num_genres = 10
genres_per_playlist = 3

simulated_playlists = []

for ind in range(num_playlists): 
    pl_indexes = numpy.random.choice(num_genres, genres_per_playlist, replace=False) # 3 genres per playlist
    genres = [list_of_genres[a] for a in pl_indexes]
    print('Generating playlists for these genres: ', genres)
    trackids = get_recommendations_tracks_from_genre(sp, genres, limit=10)
    print(len(trackids))
    simulated_playlists.append(trackids)
# print([len(a) for a in simulated_playlists])

Generating playlists for these genres:  ['rap', 'trap latino', 'dance pop']
0
Generating playlists for these genres:  ['trap', 'trap latino', 'hip hop']
0
Generating playlists for these genres:  ['pop', 'trap latino', 'pop rap']
110


## Goal: 
Given a playlist parameterized by list of track_ids, we want to 
1) generate a feature matrix of dimension num_tracks_in_playlist x num_features
2) Perform unsupervised learning to find a latent representation that maximizes the dimensions that are common to both playlists --- baseline PCA, ideal: autoencoders. 

## Function definitions for features extraction 

In [18]:
def get_genre_from_track(sp, trackid, limit=1): 
    track = sp.track(trackid)
    artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
    genres = artist["genres"]
    if len(genres): 
        return genres[0:limit]
    else: 
        return []

In [19]:
def get_audio_df(sp, trackids, numerical_features, categorical_features): 

    feature_df = pandas.DataFrame(sp.audio_features(trackids))
    feature_df = feature_df[numerical_features + categorical_features ]
    feature_df[categorical_features] = feature_df[categorical_features].astype(str)
    feature_df = pandas.get_dummies(feature_df, drop_first = True, columns = categorical_features)
    feature_df['id'] = trackids
    return feature_df

In [20]:
def get_stats(arr, col_prefix='', pvalues = [0, 25, 50, 75,100]): 
    assert isinstance(arr, numpy.ndarray) or isinstance(arr, list)
    if isinstance(arr, list): 
        arr = numpy.array(arr)
    percentiles = numpy.round(numpy.percentile(arr, pvalues, axis = 0), decimals = 2).tolist()
    pvalues = [col_prefix + '_' + str(p) for p in pvalues]
    d = dict(zip(pvalues, percentiles))
    return d

In [21]:
get_stats(numpy.random.rand(5), 'test')

{'test_0': 0.4,
 'test_25': 0.64,
 'test_50': 0.77,
 'test_75': 0.81,
 'test_100': 0.91}

In [22]:
def get_mean_variance(arr, col_prefix=''): 
    d = {}
    d[col_prefix + '_mean'] = numpy.round(numpy.mean(arr, axis = None), 2)
    d[col_prefix + '_std'] = numpy.round(numpy.std(arr, axis = None), 2)
    return d

In [23]:
def gen_basic_audio_features(sp, sample_trackid): 
    numerical_features= ['danceability','energy','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo','duration_ms']
    categorical_features = ['key', 'mode']
    audio_feats = sp.audio_features(sample_trackid)[0]
    feature = {}
    for feat in numerical_features: 
        feature[feat] = audio_feats[feat]
    for feat in categorical_features: 
        val = '_fulltrack_' + str(audio_feats[feat])
        feature[feat+val] = 1
    return feature


In [24]:
from collections import Counter
import librosa

def get_extensive_audio_features(sp, sample_trackid, pvalues= [10,25,50,75,90], perform_dsp = True, return_basic_features = False): 
    '''
    Get a single track and retrieve audio features that are in addition to the basic features.
    '''
    pitches = 'C C# D D# E F F# G G# A A# B'.split(' ')
    timbres = ['timbre_'+str(a) for a in range(12)]

    features = {'id': sample_trackid}
    
    if return_basic_features: 
        features.update(gen_basic_audio_features(sp, sample_trackid))
    
    sample_audio_analysis = sp.audio_analysis(sample_trackid)

    # processing a single track 
    num_sections = len(sample_audio_analysis['sections'])
    
    loudness = [sample_audio_analysis['sections'][a]['loudness'] for a in range(num_sections)]
    tempo = [sample_audio_analysis['sections'][a]['tempo'] for a in range(num_sections)]
    features.update(get_stats(numpy.array(loudness), 'loudness'))
    features.update(get_stats(numpy.array(tempo), 'tempo'))

    key = ['key_'+str(sample_audio_analysis['sections'][a]['key']) for a in range(num_sections)]
    mode = ['mode_'+str(sample_audio_analysis['sections'][a]['mode']) for a in range(num_sections)]
    key_counter = dict(Counter(key))
    _sum = sum(key_counter.values())
    key_counter = {a:round(b/_sum*100)/100. for a,b in key_counter.items()}
    mode_counter = dict(Counter(mode))
    _sum = sum(mode_counter.values())
    mode_counter = {a:round(b/_sum*100)/100. for a,b in mode_counter.items()}

    features.update(key_counter)
    features.update(mode_counter)


    # pitches and timbre
    num_segments = len(sample_audio_analysis['segments'])
    pitch_data = []
    timbre_data = []
    for a in range(num_segments): 
        pitch_data.append(sample_audio_analysis['segments'][a]['pitches'])
        timbre_data.append(sample_audio_analysis['segments'][a]['timbre'])
    pitch_df = pandas.DataFrame(data = numpy.array(pitch_data), columns = pitches)
    timbre_df = pandas.DataFrame(data = numpy.array(timbre_data), columns = timbres)

    for p in pitches: 
        features.update(get_stats(pitch_df[p].values, p, pvalues = [10,25,50,75,90]))
    for t in timbres: 
        features.update(get_stats(timbre_df[t].values, t, pvalues = [10,25,50,75,90]))

    if perform_dsp: 
        sample_track = sp.track(sample_trackid)
        audio_file = wget.download(sample_track['preview_url'])
        audio, sr  = librosa.load(audio_file)
        os.remove(audio_file)

        zero_crossings = librosa.feature.zero_crossing_rate(audio)
        spectral_centroids = librosa.feature.spectral_centroid(y=audio, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)

        features.update(get_mean_variance(zero_crossings, col_prefix='zero_crossings'))
        features.update(get_mean_variance(spectral_centroids, col_prefix='spectral_centroids'))
        features.update(get_mean_variance(rolloff, col_prefix='rolloff'))
        
    return features
    


In [25]:
def gen_extensive_audio_features(sp, trackids, pvalues= [10,25,50,75,90], perform_dsp = True, return_basic_features = False): 
    features_all_tracks = []
    for ind, t_id in enumerate(trackids): 
        try: 
            print('processing track ', t_id, '{f} out of {e}'.format(f=ind, e=len(trackids)))
            features_all_tracks.append(get_extensive_audio_features(sp, t_id, pvalues= [10,25,50,75,90], perform_dsp = perform_dsp, return_basic_features = return_basic_features))
        except: 
            print(traceback.print_exc(5))
            continue
    df_audio_extra = pandas.DataFrame(features_all_tracks)
    return df_audio_extra

## Testing Features on a single track 

In [26]:
trackids = get_recommendations_tracks_from_genre(sp,['classical'], limit=5)
sample_trackid = trackids[5]

In [27]:
sample_trackid
sp.audio_features(sample_trackid)


[{'danceability': 0.308,
  'energy': 0.43,
  'key': 4,
  'loudness': -15.55,
  'mode': 0,
  'speechiness': 0.0378,
  'acousticness': 0.882,
  'instrumentalness': 0.969,
  'liveness': 0.237,
  'valence': 0.531,
  'tempo': 79.67,
  'type': 'audio_features',
  'id': '7DSpITILwSijQBGJCkPRdg',
  'uri': 'spotify:track:7DSpITILwSijQBGJCkPRdg',
  'track_href': 'https://api.spotify.com/v1/tracks/7DSpITILwSijQBGJCkPRdg',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7DSpITILwSijQBGJCkPRdg',
  'duration_ms': 154467,
  'time_signature': 4}]

In [28]:
sp.audio_features(trackids)

[{'danceability': 0.125,
  'energy': 0.0382,
  'key': 4,
  'loudness': -26.221,
  'mode': 1,
  'speechiness': 0.036,
  'acousticness': 0.852,
  'instrumentalness': 0.882,
  'liveness': 0.409,
  'valence': 0.12,
  'tempo': 97.005,
  'type': 'audio_features',
  'id': '1xjHYQefL45gTbjO6rbwHC',
  'uri': 'spotify:track:1xjHYQefL45gTbjO6rbwHC',
  'track_href': 'https://api.spotify.com/v1/tracks/1xjHYQefL45gTbjO6rbwHC',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1xjHYQefL45gTbjO6rbwHC',
  'duration_ms': 165667,
  'time_signature': 4},
 {'danceability': 0.469,
  'energy': 0.0218,
  'key': 0,
  'loudness': -25.667,
  'mode': 1,
  'speechiness': 0.0328,
  'acousticness': 0.951,
  'instrumentalness': 0.865,
  'liveness': 0.163,
  'valence': 0.164,
  'tempo': 110.807,
  'type': 'audio_features',
  'id': '4G755VDRgGmg0KMGpqriQo',
  'uri': 'spotify:track:4G755VDRgGmg0KMGpqriQo',
  'track_href': 'https://api.spotify.com/v1/tracks/4G755VDRgGmg0KMGpqriQo',
  'analysis_url': 'https://a

In [29]:
numerical_features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness']
numerical_features += ['valence', 'tempo', 'duration_ms']
categorical_features = ['key', 'mode']
feature_df = get_audio_df(sp, trackids, numerical_features, categorical_features)
feature_df.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,...,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,mode_1,id
0,0.125,0.0382,-26.221,0.0360,0.852,0.882,0.4090,0.120,97.005,165667,...,0,0,1,0,0,0,0,0,1,1xjHYQefL45gTbjO6rbwHC
1,0.469,0.0218,-25.667,0.0328,0.951,0.865,0.1630,0.164,110.807,99133,...,0,0,0,0,0,0,0,0,1,4G755VDRgGmg0KMGpqriQo
2,0.526,0.0204,-27.541,0.0654,0.968,0.686,0.0526,0.367,121.287,95040,...,1,0,0,0,0,0,0,0,1,0Qvz289D59wVeN3smV25y1
3,0.413,0.2790,-19.249,0.0375,0.965,0.831,0.1180,0.580,83.303,128720,...,0,0,0,1,0,0,0,0,1,6vnzlkJDf9f3ET1IyqnsLO
4,0.518,0.2760,-18.453,0.0627,0.942,0.968,0.3140,0.272,79.406,109133,...,0,0,0,0,0,1,0,0,1,1FkTBaLsjRrQARJiPi2nba


In [30]:
gen_basic_audio_features(sp, sample_trackid)

{'danceability': 0.308,
 'energy': 0.43,
 'loudness': -15.55,
 'speechiness': 0.0378,
 'acousticness': 0.882,
 'instrumentalness': 0.969,
 'liveness': 0.237,
 'valence': 0.531,
 'tempo': 79.67,
 'duration_ms': 154467,
 'key_fulltrack_4': 1,
 'mode_fulltrack_0': 1}

In [46]:
# extensive features for sample track 

get_extensive_audio_features(sp, sample_trackid, pvalues= [10,25,50,75,90], perform_dsp = False, return_basic_features=True)


{'id': '7DSpITILwSijQBGJCkPRdg',
 'danceability': 0.308,
 'energy': 0.43,
 'loudness': -15.55,
 'speechiness': 0.0378,
 'acousticness': 0.882,
 'instrumentalness': 0.969,
 'liveness': 0.237,
 'valence': 0.531,
 'tempo': 79.67,
 'duration_ms': 154467,
 'key_fulltrack_4': 1,
 'mode_fulltrack_0': 1,
 'loudness_0': -19.35,
 'loudness_25': -16.99,
 'loudness_50': -15.45,
 'loudness_75': -14.58,
 'loudness_100': -10.63,
 'tempo_0': 76.33,
 'tempo_25': 77.07,
 'tempo_50': 77.64,
 'tempo_75': 81.24,
 'tempo_100': 83.35,
 'key_5': 0.38,
 'key_2': 0.12,
 'key_9': 0.12,
 'key_0': 0.12,
 'key_7': 0.25,
 'mode_1': 0.62,
 'mode_0': 0.38,
 'C_10': 0.11,
 'C_25': 0.18,
 'C_50': 0.39,
 'C_75': 0.84,
 'C_90': 1.0,
 'C#_10': 0.1,
 'C#_25': 0.16,
 'C#_50': 0.29,
 'C#_75': 0.5,
 'C#_90': 0.77,
 'D_10': 0.12,
 'D_25': 0.18,
 'D_50': 0.31,
 'D_75': 0.54,
 'D_90': 0.96,
 'D#_10': 0.07,
 'D#_25': 0.12,
 'D#_50': 0.21,
 'D#_75': 0.36,
 'D#_90': 0.55,
 'E_10': 0.11,
 'E_25': 0.19,
 'E_50': 0.32,
 'E_75': 0.54,
 

In [47]:
import time 
now = time.time()
testingdf= gen_extensive_audio_features(sp, trackids, pvalues= [10,25,50,75,90], perform_dsp = False, return_basic_features=True)
print(time.time()- now)

processing track  1xjHYQefL45gTbjO6rbwHC 0 out of 99
processing track  4G755VDRgGmg0KMGpqriQo 1 out of 99
processing track  0Qvz289D59wVeN3smV25y1 2 out of 99
processing track  6vnzlkJDf9f3ET1IyqnsLO 3 out of 99
processing track  1FkTBaLsjRrQARJiPi2nba 4 out of 99
processing track  7DSpITILwSijQBGJCkPRdg 5 out of 99
processing track  02g5O6VyjDuc2Z9LXCABPh 6 out of 99
processing track  6ksZzxJcKQWvMgkWKlfr0q 7 out of 99
processing track  0nBmYNulUI7WnJNAulvehe 8 out of 99
processing track  79TccKnMsqjn9LfL5uia1k 9 out of 99
processing track  6idMXLOTOGGJ9WoECQ5nKJ 10 out of 99
processing track  5ZOxtRIevqNn2ndB2Wb4iW 11 out of 99
processing track  7HSs4srn1qnZhh7WRWBVOk 12 out of 99
processing track  6I1bwLX3MeHqolxbA3Mdr8 13 out of 99
processing track  6whCwux0rqZl0X5Eeit8fJ 14 out of 99
processing track  1dunZif1ns86HFpcQ1wGQU 15 out of 99
processing track  0ASQC4HF8zDHTBmt0LfWCG 16 out of 99
processing track  74y7hBsLG8vTyntrTEYX3H 17 out of 99
processing track  7vL8vMDyh5hyqQBJEFX0

In [18]:
testingdf.head()

,id,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,...,key_fulltrack_1,key_9,key_fulltrack_11,key_fulltrack_8,key_fulltrack_7,key_fulltrack_4,key_fulltrack_0,key_fulltrack_9,key_-1,mode_-1
0,4pTB18dbtjTTCLkybdylX6,0.325,0.01080,-29.025,0.0381,0.992,0.897,0.0888,0.0408,72.487,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2FP7JttwbYPXcXxIihEvop,0.422,0.01630,-26.561,0.0399,0.989,0.918,0.1770,0.1080,64.079,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7GdYi1J7Wtmdgz0LR7uAif,0.242,0.02740,-29.170,0.0400,0.992,0.898,0.0908,0.0761,80.744,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4XsFjtQzutHM2e9EKs0NLN,0.267,0.00691,-26.649,0.0381,0.993,0.898,0.0507,0.0585,142.261,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4YBw6EAdwlm69OY2F08dEM,0.281,0.02020,-26.099,0.0395,0.993,0.910,0.1270,0.0655,77.814,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
## Testing multiprocessing library 

In [49]:
from collections import Counter
import librosa
# import multiprocess

def get_extensive_audio_features_test(sp, sample_trackid, pvalues= [10,25,50,75,90], perform_dsp = True, return_basic_features = False): 
    '''
    Get a single track and retrieve audio features that are in addition to the basic features.
    '''
    pitches = 'C C# D D# E F F# G G# A A# B'.split(' ')
    timbres = ['timbre_'+str(a) for a in range(12)]

    features = {'id': sample_trackid}
    
    if return_basic_features: 
        features.update(gen_basic_audio_features(sp, sample_trackid))
    
    sample_audio_analysis = sp.audio_analysis(sample_trackid)

    # processing a single track 
    num_sections = len(sample_audio_analysis['sections'])
    
    loudness = [sample_audio_analysis['sections'][a]['loudness'] for a in range(num_sections)]
    tempo = [sample_audio_analysis['sections'][a]['tempo'] for a in range(num_sections)]
    features.update(get_stats(numpy.array(loudness), 'loudness'))
    features.update(get_stats(numpy.array(tempo), 'tempo'))

    key = ['key_'+str(sample_audio_analysis['sections'][a]['key']) for a in range(num_sections)]
    mode = ['mode_'+str(sample_audio_analysis['sections'][a]['mode']) for a in range(num_sections)]
    key_counter = dict(Counter(key))
    _sum = sum(key_counter.values())
    key_counter = {a:round(b/_sum*100)/100. for a,b in key_counter.items()}
    mode_counter = dict(Counter(mode))
    _sum = sum(mode_counter.values())
    mode_counter = {a:round(b/_sum*100)/100. for a,b in mode_counter.items()}

    features.update(key_counter)
    features.update(mode_counter)


    # pitches and timbre
    num_segments = len(sample_audio_analysis['segments'])
    pitch_data = []
    timbre_data = []
    for a in range(num_segments): 
        pitch_data.append(sample_audio_analysis['segments'][a]['pitches'])
        timbre_data.append(sample_audio_analysis['segments'][a]['timbre'])
    pitch_df = pandas.DataFrame(data = numpy.array(pitch_data), columns = pitches)
    timbre_df = pandas.DataFrame(data = numpy.array(timbre_data), columns = timbres)

    for p in pitches: 
        features.update(get_stats(pitch_df[p].values, p, pvalues = [10,25,50,75,90]))
    for t in timbres: 
        features.update(get_stats(timbre_df[t].values, t, pvalues = [10,25,50,75,90]))

    if perform_dsp: 
        sample_track = sp.track(sample_trackid)
        audio_file = wget.download(sample_track['preview_url'])
        audio, sr  = librosa.load(audio_file)
        os.remove(audio_file)

        zero_crossings = librosa.feature.zero_crossing_rate(audio)
        spectral_centroids = librosa.feature.spectral_centroid(y=audio, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)

        features.update(get_mean_variance(zero_crossings, col_prefix='zero_crossings'))
        features.update(get_mean_variance(spectral_centroids, col_prefix='spectral_centroids'))
        features.update(get_mean_variance(rolloff, col_prefix='rolloff'))
        
    return features

now = time.time()

from joblib import parallel_backend
from joblib import Parallel, delayed
with parallel_backend('threading', n_jobs=2):
    testdicts=Parallel()(delayed(get_extensive_audio_features_test)(sp, track, pvalues,perform_dsp,return_basic_features) for track in trackids)
    
#pool = multiprocess.Pool(multiprocess.cpu_count())
#print(multiprocess.cpu_count())
#pvalues= [10,25,50,75,90]
#perform_dsp = False
#return_basic_features=True
#testdicts = [pool.apply(get_extensive_audio_features_test, args = (sp, track, pvalues,perform_dsp,return_basic_features)) for track in trackids]
#pool.close()
print(time.time()-now)

27.964767932891846


In [41]:
import math
from joblib import parallel_backend
from joblib import Parallel, delayed
with parallel_backend('threading', n_jobs=2):
    Parallel()(delayed(math.sqrt)(i ** 2) for i in range(10))

In [50]:
testdicts

[{'id': '1xjHYQefL45gTbjO6rbwHC',
  'danceability': 0.125,
  'energy': 0.0382,
  'loudness': -26.221,
  'speechiness': 0.036,
  'acousticness': 0.852,
  'instrumentalness': 0.882,
  'liveness': 0.409,
  'valence': 0.12,
  'tempo': 97.005,
  'duration_ms': 165667,
  'key_fulltrack_4': 1,
  'mode_fulltrack_1': 1,
  'loudness_0': -39.71,
  'loudness_25': -36.91,
  'loudness_50': -33.15,
  'loudness_75': -25.29,
  'loudness_100': -22.21,
  'tempo_0': 93.46,
  'tempo_25': 95.62,
  'tempo_50': 97.89,
  'tempo_75': 98.28,
  'tempo_100': 99.35,
  'key_0': 0.33,
  'key_11': 0.33,
  'key_4': 0.17,
  'key_7': 0.17,
  'mode_1': 1.0,
  'C_10': 0.06,
  'C_25': 0.11,
  'C_50': 0.27,
  'C_75': 0.53,
  'C_90': 1.0,
  'C#_10': 0.05,
  'C#_25': 0.09,
  'C#_50': 0.18,
  'C#_75': 0.37,
  'C#_90': 0.79,
  'D_10': 0.03,
  'D_25': 0.05,
  'D_50': 0.11,
  'D_75': 0.31,
  'D_90': 0.98,
  'D#_10': 0.03,
  'D#_25': 0.07,
  'D#_50': 0.14,
  'D#_75': 0.27,
  'D#_90': 0.48,
  'E_10': 0.03,
  'E_25': 0.08,
  'E_50': 

In [1]:
import multiprocessing as mp 

In [2]:
!pip install multiprocess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 KB 985.1 kB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 KB 1.2 MB/s eta 0:00:00 MB/s eta 0:00:01


In [4]:
import multiprocess
def dummy(sp, sample_trackid, pvalues= [10,25,50,75,90], perform_dsp = True, return_basic_features = False): 
    print(sp, sample_trackid, pvalues, perform_dsp, return_basic_features)


pool = multiprocess.Pool(mp.cpu_count())
print(multiprocess.cpu_count())
pvalues= [10,25,50,75,90]
perform_dsp = False
return_basic_features=True
testdicts = [pool.apply(dummy, args = (4, track, pvalues,perform_dsp,return_basic_features)) for track in range(100)]
pool.close()

4 0 [10, 25, 50, 75, 90] False True
4 1 [10, 25, 50, 75, 90] False True
4 2 [10, 25, 50, 75, 90] False True
4 3 [10, 25, 50, 75, 90] False True
4 4 [10, 25, 50, 75, 90] False True
4 5 [10, 25, 50, 75, 90] False True
4 6 [10, 25, 50, 75, 90] False True
4 7 [10, 25, 50, 75, 90] False True
4 8 [10, 25, 50, 75, 90] False True
4 98
 [10, 25, 50, 75, 90] False True
4 10 [10, 25, 50, 75, 90] False True
4 11 [10, 25, 50, 75, 90] False True
4 12 [10, 25, 50, 75, 90] False True
4 13 [10, 25, 50, 75, 90] False True
4 14 [10, 25, 50, 75, 90] False True
4 15 [10, 25, 50, 75, 90] False True
4 16 [10, 25, 50, 75, 90] False True
4 17 [10, 25, 50, 75, 90] False True
4 18 [10, 25, 50, 75, 90] False True
4 19 [10, 25, 50, 75, 90] False True
4 20 [10, 25, 50, 75, 90] False True
4 21 [10, 25, 50, 75, 90] False True
4 22 [10, 25, 50, 75, 90] False True
4 23 [10, 25, 50, 75, 90] False True
4 24 [10, 25, 50, 75, 90] False True
4 25 [10, 25, 50, 75, 90] False True
4 26 [10, 25, 50, 75, 90] False True
4 27 [10,

In [ ]:
import multiprocess
def dummy(sp, sample_trackid, pvalues= [10,25,50,75,90], perform_dsp = True, return_basic_features = False): 
    print(sp, sample_trackid, pvalues, perform_dsp, return_basic_features)


pool = multiprocess.Pool(mp.cpu_count())
print(multiprocess.cpu_count())
pvalues= [10,25,50,75,90]
perform_dsp = False
return_basic_features=True
testdicts = [pool.apply(dummy, args = (4, track, pvalues,perform_dsp,return_basic_features)) for track in range(100)]
pool.close()

In [34]:
df_audio_extra = testingdf
df_audio_extra.columns.tolist()
print(df_audio_extra.dtypes)
print(feature_df.dtypes)

id              object
loudness_0     float64
loudness_25    float64
loudness_50    float64
loudness_75    float64
                ...   
key_11         float64
key_4          float64
key_5          float64
key_0          float64
key_9          float64
Length: 145, dtype: object


NameError: name 'feature_df' is not defined

In [89]:
## use this code in case we decide to return a dataframe for basic features
feature_df = get_audio_df(sp, trackids, numerical_features, categorical_features)
df_audio_extra = gen_extensive_audio_features(sp, trackids, pvalues= [10,25,50,75,90], perform_dsp = False)
joined_df = pandas.merge(feature_df, df_audio_extra, how='inner', on='id')
print(joined_df.columns.tolist())
joined_df.head()


processing track  11WQasJcGWgh3mvR1UiBcm 0 out of 85
processing track  7hVsNjrvtfXZlzxW5vxsAi 1 out of 85
processing track  06am46cX3Z6YlSsg0TyVHA 2 out of 85
processing track  6yTPBtEggdlBgGEUL94sn1 3 out of 85
processing track  2NodH0sMaxRFSWePxkuutP 4 out of 85
processing track  7JUbQkbvdPnNqS8RdcfCtO 5 out of 85
processing track  460K7NeHx8ttnZa0Qqh4Vx 6 out of 85
processing track  1npEvrkd5GA8djI76r68Ay 7 out of 85
processing track  2bM4iYnqICtB3vAGplnz0d 8 out of 85
processing track  5jXB4p8pQu9Ahv10ibU8fG 9 out of 85
processing track  7KwH7xYAQ5rFja2GnJLIqD 10 out of 85
processing track  3eHV5kXt0OZmrUTrO8wzMX 11 out of 85
processing track  0W5hxw5mvLIVZu1QIw7IpP 12 out of 85
processing track  13rYowrPC2dk2Jhavquy1F 13 out of 85
processing track  7o5DCsdl4eIqusst2pVoKX 14 out of 85
processing track  6kHWawEbq5Vcyp8nSHhGjs 15 out of 85
processing track  2FSMEjWvRqmrdZbsqiW8ds 16 out of 85
processing track  4L9KpsmAbyhXOZZkSybfaU 17 out of 85
processing track  0CJuLeZpCdqrqoCMXWZw

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,...,key_2_y,key_7_y,key_11_y,key_4_y,key_8_y,key_0,mode_0,key_1_y,key_3_y,key_6
0,0.422,0.0461,-26.781,0.0400,0.977,0.839,0.0557,0.3370,118.922,189085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.241,0.0990,-26.281,0.0422,0.920,0.934,0.1030,0.0986,106.902,227288,...,0.08,0.25,0.17,0.08,0.08,0.08,0.42,NaN,NaN,NaN
2,0.444,0.1160,-23.172,0.0410,0.977,0.868,0.0862,0.5120,150.319,145613,...,NaN,0.33,NaN,0.67,NaN,NaN,0.50,NaN,NaN,NaN
3,0.256,0.2460,-21.403,0.0490,0.901,0.297,0.5780,0.6390,62.102,134192,...,0.38,NaN,NaN,NaN,NaN,0.25,0.12,NaN,NaN,NaN
4,0.290,0.0572,-27.498,0.0443,0.971,0.705,0.3530,0.1520,145.487,327973,...,0.19,0.12,NaN,0.19,NaN,0.12,0.25,0.06,NaN,NaN


In [90]:
## sanity 

print(len(feature_df), len(df_audio_extra), len(joined_df), len(trackids))

85 85 85 85
